In [19]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 50
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')
import seaborn as sns
import statsmodels
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import mean_squared_error

### loading in original `.csv` file and converting it a datetime df with `crime_count` column

In [2]:
df = pd.read_csv('data/extracted/crime-data-from-2010-to-present.csv')
df.head(3)

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Weapon Used Code,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
0,102005556,2010-01-25T00:00:00,2010-01-22T00:00:00,2300,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,VAN NESS,15TH,"{'latitude': '34.0454', 'needs_recoding': Fals..."
1,101822289,2010-11-11T00:00:00,2010-11-10T00:00:00,1800,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,88TH,WALL,"{'latitude': '33.9572', 'needs_recoding': Fals..."
2,101105609,2010-01-28T00:00:00,2010-01-27T00:00:00,2230,11,Northeast,1125,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,108.0,PARKING LOT,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,YORK,AVENUE 51,"{'latitude': '34.1211', 'needs_recoding': Fals..."


In [3]:
# cleaning time columns
df["Date Occurred"] = df["Date Occurred"].str.replace('T00:00:00', '')
df["Date Reported"] = df["Date Occurred"].str.replace('T00:00:00', '')
# converting them to time series
df['Date Occurred'] = pd.to_datetime(df['Date Occurred'], format= '%Y-%m-%d')
df['Date Reported'] = pd.to_datetime(df['Date Occurred'], format= '%Y-%m-%d')

In [4]:
# creating new df
time_df = df.groupby('Date Occurred').size().reset_index()
time_df = time_df.set_index('Date Occurred')
time_df = time_df.rename({0: 'crime_count'}, axis='columns')

In [5]:
time_df.head()

,crime_count
Date Occurred,
2010-01-01,2222
2010-01-02,533
2010-01-03,539
2010-01-04,558
2010-01-05,547


## Log Transforming the Unstationary Data

For dealing with stationary data, refer to `timeseries_graphs.ipynb`.

In [6]:
index = time_df.index
log_data = pd.Series(np.log(time_df['crime_count']), index=index)

In [7]:
# converting it to a dataframe
log_data = log_data.to_frame()

In [8]:
log_data.head()

,crime_count
Date Occurred,
2010-01-01,7.706163
2010-01-02,6.278521
2010-01-03,6.289716
2010-01-04,6.324359
2010-01-05,6.304449


## Train-Test Split
When building an ARIMA model, we pass in the undifferenced, logged data.

In [9]:
X = log_data.index
y = log_data['crime_count']

In [10]:
train_set = log_data.loc['2010-01-01':'2016-12-31']
test_set = log_data.loc['2017-01-01':'2018-12-31']

In [11]:
X_train, X_test = train_set.index , test_set.index
y_train, y_test = train_set['crime_count'] , test_set['crime_count']

## Model #1: Autoregression (AR) Model

In [24]:
baseline_model = AutoReg(y_train, lags=4)
output = baseline_model.fit()
print(output.summary())

# AutoReg(train_set, lags=4)

                            AutoReg Model Results                             
Dep. Variable:            crime_count   No. Observations:                 2557
Model:                     AutoReg(4)   Log Likelihood                1761.805
Method:               Conditional MLE   S.D. of innovations              0.121
Date:                Fri, 04 Dec 2020   AIC                             -4.213
Time:                        15:11:40   BIC                             -4.200
Sample:                    01-05-2010   HQIC                            -4.208
                         - 12-31-2016                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
intercept          3.2425      0.186     17.403      0.000       2.877       3.608
crime_count.L1     0.1767      0.020      8.999      0.000       0.138       0.215
crime_count.L2     0.0771      0.020

In [20]:
# predictions = baseline_model.predict(X_train)

In [21]:
# # generating RMSE
# rmse = mean_squared_error(y_train, predictions, squared=False)
# rmse

In [ ]:
# metric graphs

# output.plot_diagnostics(figsize=(10, 10))
# plt.show()

In [ ]:
# plot with forecast
# run this graph to check if the model is generating dynamic forecasts

# fig = output.plot_predict('2010-01-01', '2018-06-22', figsize=(10,6))

## Model #2: ARIMA Model

In [16]:
# arima_model = ARIMA(train_set[:-1], order=(2,0,0))
arima_model = ARIMA(y_train, order=(10,0,0), freq='D')
model_fit = arima_model.fit(disp=0)
print(model_fit.summary())

# ARIMA(y_train)

                              ARMA Model Results                              
Dep. Variable:            crime_count   No. Observations:                 2557
Model:                    ARMA(10, 0)   Log Likelihood                1752.708
Method:                       css-mle   S.D. of innovations              0.122
Date:                Fri, 04 Dec 2020   AIC                          -3481.416
Time:                        15:03:37   BIC                          -3411.257
Sample:                    01-01-2010   HQIC                         -3455.974
                         - 12-31-2016                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  6.3231      0.007    855.922      0.000       6.309       6.338
ar.L1.crime_count      0.1435      0.020      7.082      0.000       0.104       0.183
ar.L2.crime_count   

In [17]:
y_pred = arima_model.predict(X_train)

# we should be measuring this on X_test too

In [18]:
# generating RMSE
rmse = mean_squared_error(y_train, y_pred, squared=False)
rmse

1.1265687614520195e+36

In [ ]:
# plot with forecast
# run this graph to check if the model is generating dynamic forecasts

# fig = model_fit.plot_predict('2016-01-01', '2018-12-31')